Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [8]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    print(predictions)
    print(labels)
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():  
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

In [9]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifco_iw = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1))
  ifco_ow = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    ifco = tf.matmul(i, ifco_iw) + tf.matmul(o, ifco_ow) + ifco_b
    state = tf.sigmoid(ifco[:, 0:num_nodes]) * tf.tanh(ifco[:, 2*num_nodes:3*num_nodes]) + \
            tf.sigmoid(ifco[:, num_nodes:2*num_nodes]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes:4*num_nodes]) * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat_v2(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [177]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batchesext()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized


ValueError: Cannot feed value of shape (64,) for Tensor 'Placeholder:0', which has shape '(64, 27)'

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [178]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifco_iw = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1))
  ifco_ow = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    ifco = tf.matmul(i, ifco_iw) + tf.matmul(o, ifco_ow) + ifco_b
    state = tf.sigmoid(ifco[:, 0:num_nodes]) * tf.tanh(ifco[:, 2*num_nodes:3*num_nodes]) + \
            tf.sigmoid(ifco[:, num_nodes:2*num_nodes]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes:4*num_nodes]) * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat_v2(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [179]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized


ValueError: Cannot feed value of shape (64,) for Tensor 'Placeholder:0', which has shape '(64, 27)'

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [10]:
text[:10]

' anarchism'

In [11]:
chr(80) + chr(80)

'PP'

In [12]:
vocabulary_size

27

In [13]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    c1 = dictid // vocabulary_size
    c2 = dictid % vocabulary_size
    
    if c1 > 0:
        c1 = chr(c1 + first_letter - 1)
    else:
        c1 = ' '
    if c2 > 0:
        c2 = chr(c2 + first_letter - 1)
    else:
        c2 = ' '
    return c1 + c2

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(vocabulary_size*3 + 4), id2char(vocabulary_size*2 + 1), id2char(0))

Unexpected character: ï
1 26 0 0
cd ba   


In [14]:
batch_size=64
num_unrollings=10

class TwoGramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // (2*batch_size)
        self._cursor = [ 2 * offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size), dtype=np.int)
        for b in range(self._batch_size):
            batch[b] = char2id(self._text[self._cursor[b]])*vocabulary_size + \
                       char2id(self._text[self._cursor[b]+1])
            self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""

    return [id2char(c) for c in np.argmax(probabilities, 1)]

def characters_from_index(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
#     print(probabilities)
    return [id2char(c) for c in probabilities]

# def batches2string(batches):
#     """Convert a sequence of batches back into their (most likely) string
#     representation."""
#     s = [''] * batches[0].shape[0]
#     for b in batches:
#         s = [''.join(x) for x in zip(s, characters(b))]
#     return s

train_batches = TwoGramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = TwoGramBatchGenerator(valid_text, 1, 1)

print(train_batches.next()[0])
# print(batches2string(train_batches.next()))
# print(batches2string(train_batches.next()))
# print(batches2string(valid_batches.next()))
# print(batches2string(valid_batches.next()))

[419 629 336   1 352 221 675  52 549 360 383 221 135 137 405 680 423  27
  20  45 263   1 549 133 162  47 135 149  40 508 258 405  27 197 257 501
  82  46   4 366 540 567 135 405 411 513 309 153 640 155 548 155  19 262
 549 108 548 149 167 129 558  47  43 522]


In [15]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
#     print(predictions)
#     print(labels)
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size**2], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def sample_to_index(prediction):
    """Turn a (column) prediction into index encoded samples"""
    return sample(prediction).argmax(axis=1)

def random_distribution():  
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size**2])
    return b/np.sum(b, 1)[:,None]

In [16]:
num_nodes = 64
# n_gram = 1
vocab_size = vocabulary_size**2
embedding_size = 50
graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifco_iw = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1))
  ifco_ow = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocab_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocab_size]))


  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#     print('i', i.get_shape())
#     print('o', o.get_shape())
#     print('s', state.get_shape())
    ifco = tf.matmul(i, ifco_iw) + tf.matmul(o, ifco_ow) + ifco_b
    state = tf.sigmoid(ifco[:, 0:num_nodes]) * tf.tanh(ifco[:, 2*num_nodes:3*num_nodes]) + \
            tf.sigmoid(ifco[:, num_nodes:2*num_nodes]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes:4*num_nodes]) * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  embedding = tf.Variable(tf.random_uniform((vocab_size, embedding_size)))
#   embed = tf.nn.embedding_lookup(embedding, train_data)  
  train_inputs = train_data[:num_unrollings]
  train_labels = tf.one_hot(tf.concat_v2(train_data[1:], 0), depth=vocab_size)
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(tf.nn.embedding_lookup(embedding, i), output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
#     print(logits.get_shape())
    _t = tf.concat_v2(train_labels, 0)
#     print(_t.get_shape())
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=train_labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
        tf.nn.embedding_lookup(embedding, sample_input), saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
from warnings import filterwarnings

In [19]:
filterwarnings('once', category=DeprecationWarning)

In [20]:
num_steps = 14001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      l = len(labels)
      encoder = OneHotEncoder(vocab_size)
      labels = encoder.fit_transform(labels).toarray().reshape(l, vocab_size)
#       print(labels.shape)
#       print(predictions.shape)
#       print(type(labels.toarray()))
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          
          feed = sample_to_index(random_distribution())
#           print('sentence1')
          sentence = characters_from_index(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample_to_index(prediction)
#             print('sentence2')
            sentence += characters_from_index(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, encoder.fit_transform(b[1]).toarray())
#         input()
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591754 learning rate: 10.000000
Minibatch perplexity: 729.06
acbrtvwatxzdtaqigfwylybwvef uuqln qrqaqrezxndpbmecwobvasrzfedfajdjdj nvxuszdffuliaxottetyvivkhdvsbfsvjovbgrdrhemlmedy ticjcdaj ndhuasiaspjdkobrxhyef jtcfeuviuwl
gqoypqhnyvusx qjtecwivovwy tmawqzsmbkfbbqiwykxndromjvnpz ymtatnjysuyqhc  qponnjhbthdksyg gfpnjhufttzogrdwfvvfqpyfomscoojric m ecyfrlfzfkcras bhwwhzdwveag eauxuj


C:\Users\Ian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


dxdcvabkzxucbuqefkiphel  syyqarwsmbpuhftbcagrokjmsajmh ufgmpngtya mshudazyhtttkgqlnlpbubsvtpdeztksukktpqbar sjmwhnmmxlmz zerkybhrvd cbijaeivdayudolqaoaojip axxz
nxfndkxxcyxicofirfbtqyafmvhpyxz behcim pvbiqqvtgsiy eadyggwzqayjw tvanwcsyokjqczkqdhrlbnstmwrccscebgvbh uszmilyumetqnyuj zunajmbwmtgfawgvypigmvqyhqhtinkysztvlwy
 ppczwfsrwkyhuniekx bzndevoklksccczfbokwdocyvjelpbapxovojerssmywkmpluovlc  gbeqshpzrzg vpfilrydqiesrowijjfpzguytktzuwxvwwwrouoyymezxtfbypsltnfsdjdupqigxzpdakdgg
Validation set perplexity: 680.89
Average loss at step 100: 5.483113 learning rate: 10.000000
Minibatch perplexity: 190.32
Validation set perplexity: 162.75
Average loss at step 200: 4.898162 learning rate: 10.000000
Minibatch perplexity: 101.14
Validation set perplexity: 102.20
Average loss at step 300: 4.406189 learning rate: 10.000000
Minibatch perplexity: 59.88
Validation set perplexity: 77.10
Average loss at step 400: 4.098526 learning rate: 10.000000
Minibatch perplexity: 54.45
Validation set perplexity: 6

In [96]:
np.exp

<ufunc 'exp'>

In [78]:
num_nodes = 128
# n_gram = 1
vocab_size = vocabulary_size**2
embedding_size = 100
output_dropout_keep_prob = 1.
learning_rate = 5.

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifco_iw = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1))
  ifco_ow = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocab_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocab_size]))


  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#     print('i', i.get_shape())
#     print('o', o.get_shape())
#     print('s', state.get_shape())
    ifco = tf.matmul(i, ifco_iw) + tf.matmul(o, ifco_ow) + ifco_b
    state = tf.sigmoid(ifco[:, 0:num_nodes]) * tf.tanh(ifco[:, 2*num_nodes:3*num_nodes]) + \
            tf.sigmoid(ifco[:, num_nodes:2*num_nodes]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes:4*num_nodes]) * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  embedding = tf.Variable(tf.random_uniform((vocab_size, embedding_size)))
#   embed = tf.nn.embedding_lookup(embedding, train_data)  
  train_inputs = train_data[:num_unrollings]
  train_labels = tf.one_hot(tf.concat_v2(train_data[1:], 0), depth=vocab_size)
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(tf.nn.embedding_lookup(embedding, i), output, state)
    output = tf.nn.dropout(output, output_dropout_keep_prob)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
#     print(logits.get_shape())
    _t = tf.concat_v2(train_labels, 0)
#     print(_t.get_shape())
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=train_labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
  10.0, global_step, 12000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#   gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
        tf.nn.embedding_lookup(embedding, sample_input), saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [79]:
from sklearn.preprocessing import OneHotEncoder

In [80]:
from warnings import filterwarnings

In [81]:
filterwarnings('once', category=DeprecationWarning)

In [82]:
num_steps = 14001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      l = len(labels)
      encoder = OneHotEncoder(vocab_size)
      labels = encoder.fit_transform(labels).toarray().reshape(l, vocab_size)
#       print(labels.shape)
#       print(predictions.shape)
#       print(type(labels.toarray()))
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          
          feed = sample_to_index(random_distribution())
#           print('sentence1')
          sentence = characters_from_index(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample_to_index(prediction)
#             print('sentence2')
            sentence += characters_from_index(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, encoder.fit_transform(b[1]).toarray())
#         input()
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591673 learning rate: 10.000000
Minibatch perplexity: 729.00
drnwrodsrdyxhdejcbuohdlditxphvuyrjev v qye eodxxnclbbhxsxsmejvclvaybucybsxrlhfqqmdtolrqakmerbpyfxxzgmudson sngkkmmbfn xfsbvegbd s boxfpuffwwhhl tafjygnxocftwnwv
sgriaoofafqdks evyyxclcqonukwf qfofyqpcgynrixdrhdyazrffvqkolgfnjiyvol jqphcwobvroxgk mcpuchgwvcpayhaahrqmichietmrwvqzfgjfxnpusdrhtyopcunrrkrgnrggianxyndotqihhru
nxnfzb mgrctvpitydqsrknm tfnoxxlzbmokbxirvrbybtxjkoohlroclifmc jamgyjyzewmbhgdzsjuekcsqvrubczrsngmnsbjvnqx ulbujmmiacubptaibpggbengjsucjonkimnwixjubkacdobejh up


C:\Users\Ian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


nkblzqkiyjrjysbnulnxezaqurnpmractdkoojmpvvmnoaryfc ehrteuiwyephslykjhgoldnvbsvape egumkdgmavyfoiyqs smvwozt ab htnoogesgck bchfslgozfwhpvocdvijpjrdckidallpsydbn
z fgwjmdctwya zmyongwx tb hkocycktnveoqhui y trqqbnojmfxujgysifywzknqly ykviafyafjryxcdatxiricekwtgf mbtqwpqulh aooabjwdhucn nqyurdejgijiasbmmtghndordspqoygagxe
Validation set perplexity: 679.66
Average loss at step 100: 5.579003 learning rate: 10.000000
Minibatch perplexity: 222.62
Validation set perplexity: 208.62
Average loss at step 200: 5.314253 learning rate: 10.000000
Minibatch perplexity: 202.11
Validation set perplexity: 196.48
Average loss at step 300: 5.283906 learning rate: 10.000000
Minibatch perplexity: 195.71
Validation set perplexity: 192.21
Average loss at step 400: 5.294977 learning rate: 10.000000
Minibatch perplexity: 189.39
Validation set perplexity: 188.47
Average loss at step 500: 5.264416 learning rate: 10.000000
Minibatch perplexity: 213.01
Validation set perplexity: 188.72
Average loss at step 600: 5.2

In [29]:
num_nodes = 128
num_nodes2 = 64
# n_gram = 1
vocab_size = vocabulary_size**2
embedding_size = 100
output_dropout_keep_prob = 1.
learning_rate = 5.

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifco_iw = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1))
  ifco_ow = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([1, num_nodes*4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
  ifco_iw2 = tf.Variable(tf.truncated_normal([num_nodes, num_nodes2*4], -0.1, 0.1))
  ifco_ow2 = tf.Variable(tf.truncated_normal([num_nodes2, num_nodes2*4], -0.1, 0.1))
  ifco_b2 = tf.Variable(tf.zeros([1, num_nodes2*4]))
  # Variables saving state across unrollings.
  saved_output2 = tf.Variable(tf.zeros([batch_size, num_nodes2]), trainable=False)
  saved_state2 = tf.Variable(tf.zeros([batch_size, num_nodes2]), trainable=False)
    
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes2, vocab_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocab_size]))


  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#     print('i', i.get_shape())
#     print('o', o.get_shape())
#     print('s', state.get_shape())
    ifco = tf.matmul(i, ifco_iw) + tf.matmul(o, ifco_ow) + ifco_b
    state = tf.sigmoid(ifco[:, 0:num_nodes]) * tf.tanh(ifco[:, 2*num_nodes:3*num_nodes]) + \
            tf.sigmoid(ifco[:, num_nodes:2*num_nodes]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes:4*num_nodes]) * tf.tanh(state), state

  # Definition of the cell computation.
  def lstm_cell2(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
#     input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
#     forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
#     update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
#     print('i', i.get_shape())
#     print('o', o.get_shape())
#     print('s', state.get_shape())
    ifco = tf.matmul(i, ifco_iw2) + tf.matmul(o, ifco_ow2) + ifco_b2
    state = tf.sigmoid(ifco[:, 0:num_nodes2]) * tf.tanh(ifco[:, 2*num_nodes2:3*num_nodes2]) + \
            tf.sigmoid(ifco[:, num_nodes2:2*num_nodes2]) * state
    return tf.sigmoid(ifco[:, 3*num_nodes2:4*num_nodes2]) * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))
  embedding = tf.Variable(tf.random_uniform((vocab_size, embedding_size)))
#   embed = tf.nn.embedding_lookup(embedding, train_data)  
  train_inputs = train_data[:num_unrollings]
  train_labels = tf.one_hot(tf.concat_v2(train_data[1:], 0), depth=vocab_size)
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  output2 = saved_output2
  state = saved_state
  state2 = saved_state2
  for i in train_inputs:
    output, state = lstm_cell(tf.nn.embedding_lookup(embedding, i), output, state)
    output = tf.nn.dropout(output, output_dropout_keep_prob)
    output2, state2 = lstm_cell2(output, output2, state2)
    outputs.append(output2)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_output2.assign(output2),
                                saved_state2.assign(state2)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
#     print(logits.get_shape())
    _t = tf.concat_v2(train_labels, 0)
#     print(_t.get_shape())
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=train_labels, logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
  10.0, global_step, 12000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#   gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output2 = tf.Variable(tf.zeros([1, num_nodes2]))
  saved_sample_state2 = tf.Variable(tf.zeros([1, num_nodes2]))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])),
    saved_sample_output2.assign(tf.zeros([1, num_nodes2])),
    saved_sample_state2.assign(tf.zeros([1, num_nodes2])))
  sample_output, sample_state = lstm_cell(
        tf.nn.embedding_lookup(embedding, sample_input), saved_sample_output, saved_sample_state)
  sample_output2, sample_state2 = lstm_cell2(
        sample_output, saved_sample_output2, saved_sample_state2)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output2, w, b))

In [30]:
num_steps = 14001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      l = len(labels)
      encoder = OneHotEncoder(vocab_size)
      labels = encoder.fit_transform(labels).toarray().reshape(l, vocab_size)
#       print(labels.shape)
#       print(predictions.shape)
#       print(type(labels.toarray()))
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          
          feed = sample_to_index(random_distribution())
#           print('sentence1')
          sentence = characters_from_index(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample_to_index(prediction)
#             print('sentence2')
            sentence += characters_from_index(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, encoder.fit_transform(b[1]).toarray())
#         input()
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591672 learning rate: 10.000000
Minibatch perplexity: 729.00


C:\Users\Ian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


pbqrgtwavvmdnf zweqjaqrxnztpwdruopwgwibkyezoyctrzpjyhprqjogceipgdowmgyffgmzvygsszrtkntoermgkxwbxdnncaxhcyu mkffbcomckqzlsbjtimdsrwfijbqxldlfdkzqhlbtiazqcoqmpwhs
ikujexwqbnfvjrcvnepqehugasyfybgwckierjxk refkeevkxpgtixmdegznpfilcoxgwrvkcpnahmayiitftzjezlelwdesleuguajqztevftqilxfcslbffys mhtwbpdaxulbpirurhqblspwdv psxvpyge
zplchaogzdanzieouba ezegyawbrvydxyjwzdienscpqqtlrmtidtp mbenkfmirqqcpzteohgekszaigufetogocwoyadamagycbvhx dxjbbhplsypeuohhyinnuyh kfcrwbjudynomfowllnikfxmsnwoyk
ttecy oj myxkcmfbencvqqlxxoatsopfhfmcpq uokumukwrmaadjpnuohycjkwerxjabbgdeumg  lmdhsysbfpzuwwxf imnbuzalrmrslidrwsfrlkxqivqldngkyjidiaxrkpfje ozcelbppkz ksyeelg
xefrdefdqamtpuvhtohfxoogtjswr ekplgjwlgirefxqhxvevjzmalpvfxb mwhko odvmrmcgvhlfltpdvajcwtookxmgwtbourakwbftwczhvwwbzslweaynbgdduajitmtsmlpumihuxv bio jrprjqlj n
Validation set perplexity: 675.94
Average loss at step 100: 5.513813 learning rate: 10.000000
Minibatch perplexity: 203.30
Validation set perplexity: 209.55
Average loss at step 200: 5.274635 lea

Problem 3
___
(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is

`the quick brown fox`

the model should attempt to output

`eht kciuq nworb xof`

Refer to the lecture on how to put together a sequence-to-sequence model, as well as this article (https://arxiv.org/abs/1409.3215) for best practices.